In [1]:
import pandas as pd



In [23]:
#create a sample dataset for healthcare appointments no-shows prediction
data = {
    'Patient_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Gender': ['F', 'M', 'F', 'M', 'F', 'M', 'F', 'F', 'M', 'M'],
    'Age': [25, 30, 45, 50, 23, 35, 60, 40, 55, 28],
    'Scheduled_Day':(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',2023-1-11, '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10']),
    'Appointment_Day':(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05','2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11']),
    'SMS_received': [1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    'No_show': ['No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
}
    
    

In [11]:
df=pd.DataFrame(data)

# Convert Date columns safely 
df['Scheduled_Day'] = pd.to_datetime(df['Scheduled_Day'],errors='coerce')
df['Appointment_Day'] = pd.to_datetime(df['Appointment_Day'],errors='coerce')
print(df['Scheduled_Day'].unique())

#Drop rows with invalid dates
df.dropna(subset=['Scheduled_Day', 'Appointment_Day'], inplace=True)


#Add derived features
df['Waiting_Days'] = (df['Appointment_Day'] - df['Scheduled_Day']).dt.days
df['Day_Of_Week'] = df['Appointment_Day'].dt.day_name()



#Encode categorical columns
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
df['No_show'] = df['No_show'].map({'No': 0, 'Yes': 1})




<DatetimeArray>
['2023-01-01 00:00:00', '2023-01-02 00:00:00', '2023-01-03 00:00:00',
 '2023-01-04 00:00:00',                 'NaT', '2023-01-06 00:00:00',
 '2023-01-07 00:00:00', '2023-01-08 00:00:00', '2023-01-09 00:00:00',
 '2023-01-10 00:00:00']
Length: 10, dtype: datetime64[ns]


In [12]:
df.head()

,Patient_id,Gender,Age,Scheduled_Day,Appointment_Day,SMS_received,No_show,Waiting_Days,Day_Of_Week
0,1,0,25,2023-01-01,2023-01-02,1,0,1,Monday
1,2,1,30,2023-01-02,2023-01-03,0,1,1,Tuesday
2,3,0,45,2023-01-03,2023-01-04,1,0,1,Wednesday
3,4,1,50,2023-01-04,2023-01-05,1,0,1,Thursday
5,6,1,35,2023-01-06,2023-01-07,1,0,1,Saturday


In [13]:
features = ['Gender', 'Age', 'Waiting_Days', 'SMS_received']
X = df[features]
y = df['No_show']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)



DecisionTreeClassifier(random_state=42)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nclassification_report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.0

classification_report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

Confusion Matrix:
 [[0 0]
 [2 0]]


c:\Users\GAMIT PAUL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\GAMIT PAUL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\GAMIT PAUL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\GAMIT PAUL\anaconda3\Lib

In [21]:
X_test_copy = X_test.copy()
X_test_copy['Actual'] = y_test.values
X_test_copy['Predicted'] = y_pred

# Save the predictions to a CSV file